In [ ]:
# Step 0 — Mandatory environment prep (Colab)
%cd /content
!apt-get update -y
!apt-get install -y aria2

# Step 1 — Clone ComfyUI
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd /content/ComfyUI

# Step 2 — Python dependencies
!pip install -q -r requirements.txt
!pip install -q --upgrade ultralytics
!pip install -q huggingface_hub ipywidgets hf_transfer imageio-ffmpeg
%env HF_HUB_ENABLE_HF_TRANSFER=1

# Step 3 — Custom nodes: ComfyUI-Manager
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd /content/ComfyUI/custom_nodes/ComfyUI-Manager
!pip install -q -r requirements.txt

# Step 4 — Download YOLO model file used by ADetailer-like workflows
%cd /content
!mkdir -p /content/temp/models/yolov8
%cd /content/temp/models/yolov8
url="https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov9c.pt"
!aria2c -x 16 -s 16 -o face_yolov9c.pt "$url" || wget -O face_yolov9c.pt "$url"
!ls -lh

# (Optional) Run ComfyUI locally in Colab
# Note: You’ll need a tunnel (e.g., ngrok) for external access if desired.

In [ ]:
!pip install flask
!pip install pyngrok

In [ ]:
%cd /content/ComfyUI/models/checkpoints
!wget https://huggingface.co/SG161222/Realistic_Vision_V6.0_B1_noVAE/resolve/main/Realistic_Vision_V6.0_NV_B1_fp16.safetensors -O realvis.safetensors

In [ ]:
#Step 3 - repeat this step whenever you launch / re-launch Comfyui
## First put your ngrok token to the below and then run this code
## It will give a link like this at below : https://2fc5-34-134-226-xxx.ngrok-free.app
## Open it and then run web ui and once web ui started that link will start working
import os
import threading

from flask import Flask
from pyngrok import ngrok, conf
## Go to ngrok.com and get your api key, past below
conf.get_default().auth_token = "YOUR_KEY_HERE"

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)

public_url = ngrok.connect(8188).public_url
print(" * ngrok tunnel \"{}\" -> \"http://0.0.0.0:{}/\"".format(public_url, 8188))

app.config["BASE_URL"] = public_url

@app.route("/")
def index():
    return "Hello from Colab!"

threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [ ]:
%cd /content/ComfyUI
!python main.py --listen 0.0.0.0 --port 8188